<p></p><font size=8 face="courier" color="magenta">Visualize</font>

This notebook creates the widget inside of the "Visualize" section of the [Cytoscape app](../Cytoscape.ipynb).

# Setup Notebook

## Load Stuff

In [1]:
""" DEBUG """
%cd /root/Documents    
%run "Source Code/Setup Notebook.ipynb" #<-- Debug
%run "Source Code/Setup Source.ipynb"   #<-- Debug
# %cd sets the path to that of Cytoscape.ipynb


/root/Documents


## Helper: <font color="magenta">JupyterCell</font>

<p></p>
 <font color="magenta">JupyterCell</font> is a useful class that allows you to execute a specified *jupyter cell* from the command <font color="magenta">JupyterCell</font><font color="blue">.execute()</font>. This is used to change which <font color="green">widget</font> is displayed inside of the visualizer's **editor cell**.

The editor box is the thing next to the visualizer where a user can change the graph. Notice that the contents of this box changes depending on what the user clicks on the control panel. The Control Panel/ Editor Box functionality is actually a little tricky to create in jupyter and require some special things to be built.


In [2]:
class JupyterCell():
 
    def __init__(self, cell_text, js_var, text_before="# ", ):
        self.cell_text= cell_text
        self.text_before= text_before
        self.js_var = js_var
        self.search()
    
    def search(self):
        JS("""
        var cell_text= "";
        """+self.js_var+"""= 0;  
        while( ! cell_text.includes('"""+self.text_before+self.cell_text+"""')  ){
            cell_text = Jupyter.notebook.get_cell("""+self.js_var+""").get_text();  //.element["0"].innerText #<-- Old Way
            """+self.js_var+"""++
        }
        """+self.js_var+"""--
        """)
    
    def execute(self):  #<-- This gets the Cell by using the cell text. The cell text must be unique to that cell
        self.search()
        JS("""
        IPython.notebook.execute_cells(["""+self.js_var+"""])
        """)


## Create <font color="magenta">Graph</font> 

Here we create the initial <font color="magenta">Graph</font> class. This class is used to creat and manage the networks visualization. This section only consists of the initialization <font color="magenta">Graph</font> and some of it's basic functions. Most of <font color="magenta">Graph</font> functionality is actually implemented inside of the **Add Widgets** section

The <font color="magenta">Graph</font> Class is what is used to create and manage the networks visualization

In [3]:
class Graph():
    def __init__(self):
        #self.w = type("widgets",(),{})() #<-- this is where all the widgets go
        pass

# <font color="orange">Javascript Functions</font>

In [4]:
class JS_Helper():
    pass
Graph.js = JS_Helper()

## <font color="orange">start</font>

<font color="orange">start</font> creates the editable the canvas that displays the network and loads in the <font color="orange">javascript</font> helpers

 <font color="red">made width it 50 just for debugging</font>

In [5]:
def start(self):
    # We create the HTML object cy, which holds the cytoscape canvas
    display(HTML("""
    <style> .cy { width: 50vw; height: 30vw}</style>   //DEBUG: made width 50vw
    <div id="cy" class="cy"></div>
    """))
    
    # Create the JavaScript object cy, which holds all the javascript stuff related to the HTML cy
    JS("""
    var cy;
    require(['cytoscape'], function(cytoscape){       
        cy = cytoscape({ container: document.getElementById('cy'), layout: { name: 'cose' },
             style: [ { selector: 'node', style: { 'overlay-color': 'gray', 'overlay-padding': '2px', 'background-color': 'gray'  } },
                      { selector: 'edge', style: { 'overlay-color': 'gray', 'overlay-padding': '8px',
                          'line-color': 'gray', 'curve-style': 'bezier'  } } ]
        });
        
        // Here we initialize some settings for 'cy' 
        cy.io = {}       // <-- where we store all the parameters/variables we create        
        cy.maxZoom(14)   // <-- Determine how much window zooms in and out
        cy.minZoom(.4)
        
        cy.on("select", 'node, edge' , function(){     //<-- highlight node when selected
            this.style( 'overlay-opacity', .4 );
        });
        cy.on("unselect", 'node, edge' , function(){   //<-- unhighlight when not selected
            this.style( 'overlay-opacity', 0 );
        });
    });
    """)
JS_Helper.start = start

## <font color="orange">addNode</font>

This functions adds new nodes to the network when the clicks and empty part of the graph.

In [6]:
def addNode(self):
    JS("""
    cy.io.addNode = function(event){
      if( event.target === cy ){                                                          // <-- If we tapped on the background
          checkit = cy.add([ {  group: "nodes", position: event.position  } ]);          //<-- add node    
      }; 
    };
    cy.on('tap', cy.io.addNode );
    """)
JS_Helper.addNode = addNode

## <font color="orange">addEdge</font>

We add a new edge when the use double clicks one nodes and then double clicks another node. <font color="orange">addEdge</font> check for the double clicking and adds a new edge if necessary.

In [7]:
def addEdge(self):
    JS("""
    cy.io.click_hist = ["1", "2", "3", "4"];
    
    cy.io.addEdge = function(event){                                    // <-- on "tapping" canvas
        cy.io.click_hist.shift();                                         //<-- Remove oldest click

        if( event.target !== cy && event.target.isNode() ){               // <-- If we tapped a Node
          cy.io.click_hist.push( event.target.id() );                   // <-- Add node id to click history

          if( cy.io.click_hist[0]==cy.io.click_hist[1] && cy.io.click_hist[2]==cy.io.click_hist[3] ){      //<-- If 2 double clicks
              cy.add([ {  data: { source: cy.io.click_hist[0], target: cy.io.click_hist[2] }  } ]);         //<-- add edge
          };

        }else{                                                 //<-- if we didn't click a node
          cy.io.click_hist.push( String(Math.random()) );   //<-- Add random string to click history (repeats unlikely)
        }; 
    };
    cy.on("click" , cy.io.addEdge );
    """)
JS_Helper.addEdge = addEdge

## <font color="orange">editSelector</font>

<font color="orange">editSelector</font> decides what widgets are displayed inside of the editor box *("My Edit Box")*. What's displayed depends on if there are 1/many of node/edges/both.

In [8]:
def editSelector(self):
    JS("""
        var check = 0;
        var check_var;
        cy.io.select_type = "nothing"


        cy.io.test = function(event){
            cy.io.selection = cy.$(':selected')                 //<-- Get selected elements
            cy.io.last_type = cy.io.select_type

            if( cy.io.selection.length == 0  ){                 //<-- Do nothing if nothing's selected
                return
            }
            if( cy.io.selection.length == 1 ){                  //<-- If selection lenght is 1
                //DEBUG py.execute("Graph.w.id.value = '"++"'")  //<- put selectors ID in box

                if( cy.io.selection.isNode() && cy.io.last_type != "1 node" ){                      //<-- If it's a node
                    cy.io.select_type = "1 node"
                    //DEBUG py.execute("Graph.editor_box.widg = Graph.box.show('1 node')")
                    IPython.notebook.execute_cells([edit_num])  //<-- edit_num was setup by the Graph.editor_box 
                }
                if( cy.io.selection.isEdge() &&  cy.io.last_type != "1 edge" ){                     //<-- If it's an edge
                    cy.io.select_type = "1 edge"
                    //DEBUG py.execute("Graph.editor_box.widg = Graph.box.show('1 edge')")
                    IPython.notebook.execute_cells([edit_num])  //<-- edit_num was setup by the Graph.editor_box
                }

            } else {                                           //<-- If Many elements
                console.log("many items")
            }

        };

        cy.on('select unselect', 'node, edge', cy.io.test );
    """)
JS_Helper.editSelector = editSelector

##  <font color="teal">Style</font> <font color="orange">Functions</font>
The following code sets up the <font color="orange">javascript</font> functions/variables that handle the style nodes and edges

Create object that holds style stuff. Add to it the default style of nodes and edges

In [ ]:
JS("""


""")

In [141]:
### <font color="teal">Style</font> 

In [ ]:
for (var property in saved_style ) {
    if( saved_style[property] != "NaNpx" && !property.includes("pie") ){  // this removes error messages caused by weird stuff in cytoscape.js
        cy.io.selection[0].style( property, saved_style[property] )
    }
}

# <font color="green">Widgets</font> 

The purpose of the <font color="green">widgets</font>  section is to create all the basic "tools" that are used to manipulate the cytoscape canvas. We create the tools and the functions that allow them to do their job. Later, in the **"Display Boxes"** section, we combine to the tools together to build a pretty layout

In [9]:
Graph.w = type("widget_list",(),{})()
Graph.wf = type("widget_functions",(),{})()
Graph.w.time = 0 #<-- variables used to measure time between update events (see size for example)

## <font color="green">Delete</font> 

In [10]:
Graph.w.delete = ipw.Button(description='Delete',button_style='danger')
#Graph.w.delete

In [11]:
def delete(b):
    JS("cy.io.last_deleted = cy.$(':selected').remove();")   #<-- delete elements
    Graph.w.undo.disabled = False
Graph.wf.delete = delete
Graph.w.delete.on_click( Graph.wf.delete )

## <font color="green">Undo Delete </font> 

In [12]:
Graph.w.undo = ipw.Button(description='Undo',button_style='primary', disabled= True )
#Graph.w.undo

In [13]:
def undo(b):
    JS(" cy.add( cy.io.last_deleted );")   #<-- Add last deleted element
    b.disabled = True
Graph.wf.undo = undo
Graph.w.undo.on_click( Graph.wf.undo )

## <font color="green">Color </font> 

In [28]:
Graph.w.color = ipw.ColorPicker(  )
#Graph.w.color

In [29]:
Graph.w.color.time = 0
def color(b):
    if time.time() - Graph.w.color.time > .05:  #<-- make function run only ever 1/10 a second
        Graph.w.color.time = time.time()
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isNode() ){
                ele.style( "background-color", '"""+Graph.w.color.value+"""' )
            } else{
                ele.style( "line-color", '"""+Graph.w.color.value+"""' )
            }
        }
        """)          
Graph.wf.color = color
Graph.w.color.observe( Graph.wf.color )

## <font color="green">Background Color </font> 

In [30]:
Graph.w.background = ipw.Button( button_style='danger', icon='image', layout= ipw.Layout(width='auto') )
#Graph.w.background

In [31]:
def background(b):
    JS(f"document.getElementById('cy').style.backgroundColor = '{Graph.w.color.value}'")  
Graph.wf.background = background
Graph.w.background.on_click( Graph.wf.background )

## <font color="green">Size</font> 

In [32]:
Graph.w.size = ipw.IntSlider(value=30, min=1, max=200, continuous_update=True )
#Graph.w.size 

In [33]:
def size(b):
    if time.time() - Graph.w.time > .05:  #<-- make function run only ever 1/10 a second
        Graph.w.time = time.time()
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isNode() ){
                ele.style( "width", """+str(Graph.w.size.value)+""" )
                ele.style( "height", """+str(Graph.w.size.value)+""" )
            } else{
                ele.style( "width", """+str(Graph.w.size.value/6)+""" )
            }
        }
        """)           
Graph.wf.size = size
Graph.w.size.observe( Graph.wf.size )

## <font color="green">Opacity</font>  

In [34]:
Graph.w.opacity = ipw.FloatSlider(value=.5, min=0, max=1, step=.05, continuous_update=True )
#Graph.w.opacity 


In [35]:
def opacity(b):
    if time.time() - Graph.w.time > .05:  #<-- make function run only ever 1/10 a second
        Graph.w.time = time.time()
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            cy.io.selection[ i ].style( "opacity", """+str(Graph.w.opacity.value)+""" )
        }
        """)           
Graph.wf.opacity = opacity
Graph.w.opacity.observe( Graph.wf.opacity )

## <font color="green">ID Text</font> 

In [36]:
Graph.w.id = widgets.Text( description='ID :', continous_update = False  )#, continous_update = False )
#Graph.w.id

In [37]:
def setId(b):
    JS("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isNode() ){
            ele.style( "width", """+str(Graph.w.size.value)+""" )
            ele.style( "height", """+str(Graph.w.size.value)+""" )
        } else{
            ele.style( "width", """+str(Graph.w.size.value/6)+""" )
        }
    }
    """)           
Graph.wf.setId = setId
Graph.w.id.observe( Graph.wf.setId )

## <font color="green">Search</font> 

In [38]:
Graph.w.search = widgets.Text( description='Search :', continuous_update=False )
#Graph.w.search

In [39]:
def search(b):
    JS(f"""
    cy.$(':selected').unselect()
    cy.$('{Graph.w.search.value}').select()
    """)   #<-- delete elements
Graph.wf.search = search
Graph.w.search.observe( Graph.wf.search )

## <font color="green">Search Select</font>

In [40]:
Graph.w.searchSelect = ipw.ToggleButton( icon='eye', layout= ipw.Layout(width='auto')  )
#Graph.w.searchSelect

In [41]:
def searchSelect(b):
    
    if Graph.w.searchSelect.value:   #<-- when activated
        Graph.w.searchSelect.icon = 'eye-slash'
        JS("""
        cy.io.to_hide = cy.elements().difference( cy.io.selection )
        cy.io.to_hide.style("visibility", "hidden")
        """)
    
    else: #<-- when not clicked
        Graph.w.searchSelect.icon = 'eye'
        JS("""cy.io.to_hide.style("visibility", "visible")""")
        
Graph.wf.searchSelect = searchSelect
Graph.w.searchSelect.observe( Graph.wf.searchSelect )



## <font color="green">Z-Index Labels Behind/In-Front</font> 

In [42]:
Graph.w.z_index = ipw.IntSlider(value=0, min=0, max=5, continuous_update=True )
#Graph.w.z_index

In [43]:
def z_index(b):
    if time.time() - Graph.w.time > .05:  #<-- make function run only ever 1/10 a second
        Graph.w.time = time.time()
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            ele.style("z-index", '"""+str(Graph.w.z_index.value)+"""' )
        }
        """)               
Graph.wf.z_index = z_index
Graph.w.z_index.observe( Graph.wf.z_index )

##  <font color="salmon">Label</font> <font color="green">Widgets</font> 

### <font color="salmon">Label Text</font>

In [44]:
Graph.w.label = widgets.Text( continuous_update=False )
#Graph.w.label

In [45]:
def label(b):
    if time.time() - Graph.w.time > .05:  #<-- make function run only ever 1/10 a second
        Graph.w.time = time.time()
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            ele.style("label", '"""+Graph.w.label.value+"""' )
        }
        """)           
Graph.wf.label = label
Graph.w.label.observe( Graph.wf.label )

### <font color="salmon">Label Color</font>

In [46]:
Graph.w.label_color = ipw.ColorPicker( )
#Graph.w.label_color

In [47]:
def label_color(b):
    JS("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        ele.style( "color", '"""+Graph.w.label_color.value+"""' )
    }
    """)          
Graph.wf.label_color = label_color
Graph.w.label_color.observe( Graph.wf.label_color )

### <font color="salmon">Label Size</font>

In [48]:
Graph.w.label_size = ipw.IntSlider(value=16, min=6, max=66)
#Graph.w.label_size

In [49]:
def label_size(b):
    JS("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        ele.style( "font-size", '"""+str(Graph.w.label_size.value)+"""px' )
    }
    """)          
Graph.wf.label_size = label_size
Graph.w.label_size.observe( Graph.wf.label_size )

### <font color="salmon">Node Label Position</font>  

In [50]:
Graph.w.label_pos = ipw.Dropdown( options=['center','top','bottom','left','right'] )
#Graph.w.shape

In [51]:
def label_pos(b):
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            var label_pos = '"""+Graph.w.label_pos.value+"""'
            if( ele.isNode() ){
                if( label_pos == 'center'  ){
                    ele.style( "text-halign", 'center' )
                    ele.style( "text-valign", 'center' )  
                    
                }else if( label_pos=='top' || label_pos=='bottom'){
                    ele.style( "text-halign", 'center' )
                    ele.style( "text-valign", label_pos )      
                    
                }else{
                    ele.style( "text-halign",  label_pos )
                    ele.style( "text-valign", 'center' )                  
                }

            } 
        }
        """)           
Graph.wf.label_pos = label_pos
Graph.w.label_pos.observe( Graph.wf.label_pos )

## <font color="DarkKhaki">Node</font> <font color="green">Widgets</font> 

### <font color="DarkKhaki">Node Shape</font> 

In [52]:
Graph.w.shape = ipw.Dropdown( options=['ellipse','triangle','rectangle','rhomboid','diamond','pentagon','hexagon',
                                       'concavehexagon','heptagon','octagon','star','tag','vee'], continuous_update=True )
#Graph.w.shape

In [53]:
def shape(b):
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isNode() ){
                ele.style( "shape", '"""+Graph.w.shape.value+"""' )
            } 
        }
        """)           
Graph.wf.shape = shape
Graph.w.shape.observe( Graph.wf.shape )

## <font color="purple">Edge</font> <font color="green">Widgets</font> 

### <font color="purple">Edge Arrow Start</font> 

In [54]:
Graph.w.start = ipw.Dropdown( options=['none','triangle','tee','diamond','square','circle','vee','triangle-tee'] )
#Graph.w.start

In [55]:
def start(b):
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isEdge() ){
                ele.style( "source-arrow-color", ele.style("line-color")  )
                ele.style( "source-arrow-shape", '"""+Graph.w.start.value+"""' )
            } 
        }
        """)           
Graph.wf.start = start
Graph.w.start.observe( Graph.wf.start )

### <font color="purple">Edge Arrow End</font> 

In [56]:
Graph.w.end = ipw.Dropdown( options=['none','triangle','tee','diamond','square','circle','vee','triangle-tee'] )
#Graph.w.end

In [57]:
def end(b):
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isEdge() ){
                ele.style( "target-arrow-color", ele.style("line-color")  )
                ele.style( "target-arrow-shape", '"""+Graph.w.end.value+"""' )
            } 
        }
        """)           
Graph.wf.end = end
Graph.w.end.observe( Graph.wf.end )

### <font color="purple">Edge Line Style</font> 

In [58]:
Graph.w.lineStyle = ipw.Dropdown( options=['solid','dotted','dashed'] )
#Graph.w.lineStyle

In [60]:
def lineStyle(b):
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isEdge() ){
                ele.style( "lineStyle", '"""+Graph.w.lineStyle.value+"""' )
            } 
        }
        """)           
Graph.wf.lineStyle = lineStyle
Graph.w.lineStyle.observe( Graph.wf.lineStyle )

# <font color="orchid">Display Boxes</font>

Here we take the prviously created <font color="green">widgets</font> and we group them rogether into a pretty box.

In [89]:
Graph.box = type("box",(),{})()

**Padding : ** \[ top/ right/ bottom/ left \]

## Style Box

We add some style/layout properties to the widgets we will use in the style box

In [90]:
Graph.box.color = ipw.HBox([ ipw.Label('Color :'), Graph.w.color, 
                            ipw.Label('    '), Graph.w.background ])

In [91]:
Graph.box.size = ipw.HBox([ipw.Label('Size :'), Graph.w.size ])

In [92]:
Graph.box.shape = ipw.HBox([ipw.Label('Shape :'), Graph.w.shape ])
Graph.box.shape.layout.padding = '10px 0px 0px 0px'

In [93]:
Graph.box.lineStyle = ipw.HBox([ipw.Label('Line Style :'), Graph.w.lineStyle ])
Graph.box.lineStyle.layout.padding = '10px 0px 0px 0px'

In [94]:
Graph.box.opacity = ipw.HBox([ipw.Label('See-Through :'), Graph.w.opacity ])
Graph.box.opacity.layout.padding = '10px 0px 0px 0px'

In [95]:
Graph.box.edge_start = ipw.HBox([ipw.Label('Edge Start :'), Graph.w.start ])
Graph.box.edge_start.layout.padding = '10px 0px 0px 0px'

In [96]:
Graph.box.edge_end = ipw.HBox([ipw.Label('Edge End :'), Graph.w.end ])
Graph.box.edge_end.layout.padding = '10px 0px 0px 0px'

In [97]:
Graph.box.z_index = ipw.HBox([ipw.Label('Behind/In-Front :'), Graph.w.z_index ])
Graph.box.z_index.layout.padding = '10px 0px 0px 0px'

Now we create the style box

In [98]:
Graph.box.style = ipw.VBox([Graph.box.size,
                            Graph.box.color,
                            Graph.box.opacity,
                            Graph.box.shape,
                            Graph.box.lineStyle,
                            Graph.box.edge_start,
                            Graph.box.edge_end,
                            Graph.box.z_index,
                           ])
#Graph.box.style

## Label Box

We add some style/layout properties to the widgets we will use in the label box

In [99]:
Graph.box.label_text = ipw.HBox([ipw.Label('Text :'), Graph.w.label ])

In [100]:
Graph.box.label_color = ipw.HBox([ipw.Label('Color :'), Graph.w.label_color ])

In [101]:
Graph.box.label_size = ipw.HBox([ipw.Label('Size :'), Graph.w.label_size ])
Graph.box.shape.layout.padding = '10px 0px 0px 0px'

In [104]:
Graph.box.label_pos = ipw.HBox([ipw.Label('Position :'), Graph.w.label_pos ])
Graph.box.label_pos.layout.padding = '10px 0px 0px 0px'

Create the label box

In [105]:
Graph.box.label = ipw.VBox([Graph.box.label_text,
                            ipw.HTML("<hr style='padding: 2px; margin: 2px'>"),
                            Graph.box.label_color,
                            Graph.box.label_size,
                            Graph.box.label_pos,
                           ])
#Graph.box.label

## Variables Box

In [106]:
Graph.box.vars = ipw.HTML("<h4>Not implemented yet</h4>")

## Combine Style, Label, Variables

In [107]:
Graph.box.combo = widgets.Accordion(children=[ Graph.box.style ,Graph.box.label, Graph.box.vars])
Graph.box.combo.set_title(0, 'Style')
Graph.box.combo.set_title(1, 'Label')
Graph.box.combo.set_title(2, 'Variables')
#Graph.box.combo

## End Box

In [108]:
Graph.box.search = ipw.HBox([ Graph.w.search, Graph.w.searchSelect, ipw.HTML("<p>&nbsp&nbsp&nbsp&nbsp&nbsp</p>") ])
Graph.box.search.layout.justify_content = "center"
Graph.box.search.layout.padding = '10px 0px 0px 0px'
#Graph.box.search

In [109]:
Graph.box.delete = ipw.HBox([ Graph.w.delete, Graph.w.undo ] )
Graph.box.delete.layout.justify_content = "center"
Graph.box.delete.layout.padding = '10px 0px 0px 0px'
#Graph.box.delete

In [110]:
Graph.box.end = ipw.VBox([Graph.box.search, 
                          Graph.box.delete ])
#Graph.box.end

## Box of Everything

In [116]:
Graph.box.all = ipw.VBox([ Graph.box.combo,
                           Graph.box.end])
#Graph.box.all

# Create Graph

##  create graph and editor

Must re-execute this when debugging code

In [117]:
cy = Graph()

Graph.canvas_box = JupyterCell("My Canvas Box", "canvas_num")
Graph.canvas_box.execute()


In [119]:
Graph.editor_box = JupyterCell("My Edit Box", "edit_num")
#Graph.editor_box.widg = ipw.HTML("<h5> </h5>") #<-- widget initially empty
Graph.editor_box.execute()


In [121]:
cy.js.addNode()
cy.js.addEdge()
cy.js.editSelector()

In [ ]:
JS("""
//cy.removeListener('tap', handler);
//cy.removeListener( "click" ) 
//cy.removeListener( "tap" )
//cy.removeListener( "select" )
//cy.removeListener( "unselect" )
""")

In [ ]:
#def selectColor(self):

#checkit = cy.io.selection
#checkit.style("overlay-color","gray")
#checkit.style("overlay-opacity",".3")
#checkit.style("overlay-padding","8px")

In [ ]:
JS(f"document.getElementById('cy').style.backgroundColor = '{Graph.w.color.value}'")  

In [ ]:
Graph.editor_box.widg = Graph.box.show('1 node')

# Visualize (Mock Setup)

In [128]:
# My Edit Box
display( Graph.box.all )



In [118]:
# My Canvas Box
cy.js.start()

In [ ]:
i=0

In [ ]:
JS("""
    var ele = cy.io.selection[i]
    ele.style("z-index", '"""+str(Graph.w.z_index.value)+"""' )
""")

In [ ]:
JS("""
    var check = 0;
    var check_var;

    cy.io.test = function(event){
        cy.io.selection = cy.$(':selected')
        
        check += 1
        console.log( String(check) +": selection changed")
        check_var = 
        cehck_var2 = event
    };

    cy.on('select unselect', 'node, edge', cy.io.test );
""")


In [ ]:
#import inspect
#lines = inspect.getsource(cy.js.addEdge)
#print(lines)

In [ ]:
"""
JS(

cyto.layout({ name: 'circle'}).run();

)
"""

# Scratch Paper

In [140]:
%%js
for (var property in saved_style ) {
    if( saved_style[property] != "NaNpx" && !property.includes("pie") ){  // this removes error messages caused by weird stuff in cytoscape.js
        cy.io.selection[0].style( property, saved_style[property] )
    }
}

<IPython.core.display.Javascript object>

In [132]:
%%js
Object.keys(saved_style3).forEach(function(key,index) {
    console.log(key,index)
    // key: the name of the object key
    // index: the ordinal position of the key within the object 
});

<IPython.core.display.Javascript object>